In [1]:
import os
import sys

    
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from PIL import Image, ImageDraw

import radii as radi
import SimpleITK as sitk
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import Interface as I
from getting_started import getting_started_dir


inputTifImage_s13 = I.os.path.join(getting_started_dir, 'radii/data/neuron1/tif/max_z_projections','S13_max_z_projection_croped.tif')

/home/amir/Projects/in_silico_framework_env/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


trying to connect to distributed locking server {'config': {'socket_timeout': 1, 'host': 'spock', 'port': 8885}, 'type': 'redis'}
trying to connect to distributed locking server {'config': {'socket_timeout': 1, 'host': 'localhost', 'port': 6379}, 'type': 'redis'}
success!
Current version: v0.0.1+373.gac1f7d4.dirty
Current pid: 32408
Loading mechanisms:
setting up local multiprocessing framework ... done


The loaded modules with __version__ attribute are:
IPython: 5.1.0, Interface: v0.0.1+373.gac1f7d4.dirty, SocketServer: 0.4, Tkinter: $Revision: 81008 $, _ast: 82160, _csv: 1.0, _ctypes: 1.1.0, _curses: 2.2, _struct: 0.2, argparse: 1.1, ast: 82160, blosc: 1.5.1, bluepyopt: 1.8.21, bottleneck: 1.1.0, cPickle: 1.71, cffi: 1.7.0, cgi: 2.6, chest: 0.2.3, click: 6.6, cloudpickle: 0.2.2, colorama: 0.3.7, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.8.0, dask: 0.18.2, dateutil: 2.5.3, deap: 1.2, decimal: 1.70, decorator: 4.0.10, distributed: 1.22.1, distutils: 2.7.12, email: 4.0.3, fut

/home/amir/Projects/in_silico_framework/Interface.py:172: UserWarning: The source folder has uncommited changes!
  if get_versions()['dirty']: warnings.warn('The source folder has uncommited changes!')


In [2]:
imageFileReader = sitk.ImageFileReader()
imageFileReader.SetFileName(inputTifImage_s13)
inputImage_s13 = imageFileReader.Execute()

In [3]:
radiusCalculator = radi.calcRad.RadiusCalculator(xyResolution=0.092,
                                                 zResolution=0.5,
                                                 xySize=20,
                                                 numberOfRays=10,
                                                 tresholdPercentage=0.25)


In [15]:
def drawLine(image, rays):
    
    im = Image.open(image).convert("RGB")
    draw = ImageDraw.Draw(im) 
 
    #draw.line((backCounterPoint[0], backCounterPoint[1], frontCounterPoint[0], frontCounterPoint[1]), fill=200, width=5)
    for rayIndex, ray in enumerate(rays):
        draw.text((0,0),"lines next to the numbers are showing front rays",fill=(255,255,255,128))
        draw.text((ray[-1][0], ray[-1][1]),str(rayIndex),fill=(255,255,255,128))
        draw.line((ray[0][0], ray[0][1], ray[-1][0], ray[-1][1]),fill="#ff0000", width=2)
    im.show()

    #im.save(sys.stdout, "PNG")
    

In [21]:
def plotCircularProfiles(raysProfileList):
    

 #   t1 = np.arange(0.0, 5.0, 0.1)
    length = len(raysProfileList)
    #plt.figure(1)
    plt.figure(figsize=(10,18))
    
    for rayIndex, ray in enumerate(raysProfileList[0:length]):
        
        plt.subplot(len(raysProfileList),1, rayIndex+1)
        
        #plt.subplot(len(raysProfileList),1, rayIndex+1, frameon=False)
        #plt.axis('off')  
        plt.plot(ray,label=str(rayIndex))
        plt.legend()
        plt.tight_layout()
    plt.show() 


In [31]:

imgArray = sitk.GetArrayViewFromImage(inputImage_s13)

f,imgplot=plt.subplots()
imgplot = plt.imshow(imgArray);

pos = []
def onclick(event):
    pos = []
    pos.append([event.xdata,event.ydata])
    statistics(pos[0])

f.canvas.mpl_connect('button_press_event', onclick)
f.show()

In [30]:
def statistics(pos):
    rays, raysProfiles = radiusCalculator.getProfileAtThisPoint(inputImage_s13, pos)
    
    selectedBackProfile, selectedFrontProfile, radiusList,\
    minRadius, BackCountorPoint, frontCOunterPoint,\
    counterList,raysProfiles_1 = radiusCalculator.getRadiusFromProfile(inputImage_s13, pos)
    
    drawLine(inputTifImage_s13, rays)
    plotCircularProfiles(raysProfiles)